In [1]:
import pandas as pd

In [2]:
cd /content/drive/MyDrive/ARPIX/ReliableQA/build_dataset

/content/drive/MyDrive/ARPIX/ReliableQA/build_dataset


In [3]:
pwd

'/content/drive/MyDrive/ARPIX/ReliableQA/build_dataset'

## manually 제작한 초기 template 불러오기

In [4]:
init_dataset = pd.read_csv('RQA_init_dataset.csv', encoding='cp949')

In [5]:
init_dataset.head()

,class,fine_class,text,target
0,TRS,K2E,영어로 고양이가 뭐에요?,고양이
1,TRS,K2E,영어로 기린이 뭐에요?,기린
2,TRS,K2E,고양이가 영어로 뭐에요?,고양이
3,TRS,K2E,기린이 영어로 뭐에요?,기린
4,TRS,M2E,고양이의 의미가 뭐에요?,고양이


In [6]:
text = init_dataset['text'].tolist()
print(len(text))
print(text)

35
['영어로 고양이가 뭐에요?', '영어로 기린이 뭐에요?', '고양이가 영어로 뭐에요?', '기린이 영어로 뭐에요?', '고양이의 의미가 뭐에요?', '고양이는 무슨 뜻이에요?', '기린은 무슨 뜻이에요?', 'What is cat in English?', 'What is cat in Korean?', 'What does cat mean?', 'How do you use cat in a sentence?', 'Can you use cat in a sentence?', '고양이 예문 만들어 주세요.', '고양이로 예문 만들어 주실 수 있나요?', '기린으로 예문 만들어 주실 있어요?', 'How do you say cat?', '고양이 발음 어떻게 해요?', 'How do you say cat in English?', 'Can you pronounce cat?', 'How do you pronounce cat?', 'Where is the stress in cat?', '고양이는 어떻게 말해요?', '기린은 어떻게 말해요?', 'How do you spell cat?', '고양이의 철자가 뭐에요?', '고양이는 어떻게 써요?', '기린은 어떻게 써요?', 'Can you repeat that, please?', 'Can you say that again, please?', 'Pardon me?', 'Can you repeat that?', 'Can you say that again?', '다시 말해 주실 수 있나요?', '잘 못 들었어요.', '반복해 주실 수 있나요']


## paraphrasing engine 활용해 template augmentation

In [ ]:
pip install pororo

In [8]:
import pororo
from pororo import Pororo

In [ ]:
pg = Pororo(task="pg", lang="ko")

In [10]:
pg.predict(text[0])

'영어로 고양이는 뭐야?'

In [11]:
model = pg._model
example = pg._preprocess(text[0])
output = model.translate(example, nbest=5)
output = pg._postprocess(output)
output

'영어로 고양이는 뭐야?'

In [12]:
encoded = model.encode(example)
outputs = pg._model.generate(encoded, beam=3, nbest=3, verbose=True)
[pg._postprocess(model.decode(x['tokens'])) for x in outputs]

['영어로 고양이는 무엇인가?', '영어로 고양이는 뭐야?', '영어로 고양이가 뭐야?']

In [13]:
def generate_paraphrases(transformer, sentence, n_outputs):
    model = transformer._model
    preprocessed = transformer._preprocess(sentence)
    encoded = model.encode(preprocessed)
    outputs = model.generate(encoded, sampling=False, beam=n_outputs, nbest=n_outputs)
    scores = [x['score'] for x in outputs]
    outputs = [transformer._postprocess(model.decode(x['tokens'])) for x in outputs]
    return outputs, scores


# outputs, scores = generate_paraphrases(pg, "고양이는 무슨 뜻이에요?", 5)
# print(outputs)

In [14]:
pg_en = Pororo(task='pg', lang='en')
generate_paraphrases(pg_en, 'What is "cat" in English?', 5)

(['What\'s " cat " in English?',
  'What\'s "cat " in English?',
  "What's a cat in English?",
  "What's Cat in English?",
  'What is " cat " in English?'],
 [tensor(-0.2925),
  tensor(-0.3128),
  tensor(-0.3400),
  tensor(-0.4512),
  tensor(-0.5330)])

In [15]:
def isKorean(text):
    word = text[0]
    if word.encode().isalpha():
        return False
    else:
        return True

isKorean('안녕하세요')

True

In [16]:
import tqdm 
from tqdm import tqdm

In [17]:
def pg_augmentation(dataset, n_outputs):
    result = []

    pg_ko = Pororo(task='pg', lang='ko')
    pg_en = Pororo(task='pg', lang='en')

    for sentence in tqdm(dataset):
        if isKorean(sentence):
            augmented, _ = generate_paraphrases(pg_ko, sentence, n_outputs)
            result.append(sentence)
            result.extend(augmented)
        else:
            augmented, _ = generate_paraphrases(pg_en, sentence, n_outputs)
            result.append(sentence)
            result.extend(augmented)
    
    return result

In [18]:
augmented_data = pg_augmentation(text, 3)

100%|██████████| 35/35 [00:58<00:00,  1.69s/it]


In [19]:
len(augmented_data)

140

## augmented template과 기존 template 합치기

In [20]:
df = init_dataset.copy()
for i in range(len(text)):
    result = []
    for j in range(1, 4):
        idx = 4*i + j
        new_data = {
            'class' : init_dataset.loc[i]['class'],
            'fine_class' : init_dataset.loc[i]['fine_class'],
            'text' : augmented_data[idx],
            'target' : init_dataset.loc[i]['target']
        }

        temp1 = df[df.index < idx]
        temp2 = df[df.index >= idx]
        df = temp1.append(new_data,ignore_index=True).append(temp2, ignore_index=True)


In [21]:
df.loc[:]

,class,fine_class,text,target
0,TRS,K2E,영어로 고양이가 뭐에요?,고양이
1,TRS,K2E,영어로 고양이는 무엇인가?,고양이
2,TRS,K2E,영어로 고양이는 뭐야?,고양이
3,TRS,K2E,영어로 고양이가 뭐야?,고양이
4,TRS,K2E,영어로 기린이 뭐에요?,기린
...,...,...,...,...
135,RPT,K2E,잘 못 들었어요.,RPT
136,RPT,K2E,반복해 주실 수 있나요,RPT
137,RPT,K2E,반복해 주시겠어요?,RPT
138,RPT,K2E,반복해 주시겠습니까?,RPT


In [ ]:
df = df.drop([30, 31, 34, 43, 46, 47, 53, 54, 63, 94, 102, 106, 107])  # 이상한 template 수기로 삭제

In [23]:
df.index = range(len(df))

In [24]:
df.loc[:]

,class,fine_class,text,target
0,TRS,K2E,영어로 고양이가 뭐에요?,고양이
1,TRS,K2E,영어로 고양이는 무엇인가?,고양이
2,TRS,K2E,영어로 고양이는 뭐야?,고양이
3,TRS,K2E,영어로 고양이가 뭐야?,고양이
4,TRS,K2E,영어로 기린이 뭐에요?,기린
...,...,...,...,...
122,RPT,K2E,잘 못 들었어요.,RPT
123,RPT,K2E,반복해 주실 수 있나요,RPT
124,RPT,K2E,반복해 주시겠어요?,RPT
125,RPT,K2E,반복해 주시겠습니까?,RPT


## 영어 단어 리스트 불러오기

In [25]:
# http://dr-roach.com/blog/korean-postposition/ 에서 가져온 코드

import re
def ends_with_jong(kstr):  # 받침 유무를 확인하는 함수
    k = kstr[-1]
    if "가" <= k <= "힣":
        return (ord(k)-ord("가")) % 28 > 0
    else:
        return

In [26]:
ends_with_jong("기린")

True

In [27]:
test_sentence = "cat의 철자가 뭐에요"
target_word = "고양이"
test_sentence.find(target_word)

-1

In [28]:
def is_postposition(text, target_word):  # 문장의 타겟 단어 뒤에 조사가 있는지 조사
    start_idx = text.find(target_word)
    target_idx = start_idx + len(target_word)
    if text[target_idx] == ' ':
        return False
    else:
        return text[target_idx]

is_postposition("cat의 뜻이 뭐야?", 'cat')

'의'

In [29]:
vocab_text = open("en_vocab.txt", "r")  # https://www.eslkidstuff.com/WordLists.htm
  
# reading the file
data = vocab_text.read()
# replacing end splitting the text 
# when newline ('\n') is seen.
data_into_list = data.split(" \t")
print(data_into_list)
vocab_text.close()

['1', 'the', '76', 'more', '151', 'sentence', '226', 'between\n2', 'of', '77', 'day', '152', 'set', '227', 'city\n3', 'to', '78', 'could', '153', 'three', '228', 'tree\n4', 'and', '79', 'go', '154', 'want', '229', 'cross\n5', 'a', '80', 'come', '155', 'air', '230', 'since\n6', 'in', '81', 'did', '156', 'well', '231', 'hard\n7', 'is', '82', 'my', '157', 'also', '232', 'start\n8', 'it', '83', 'sound', '158', 'play', '233', 'might\n9', 'you', '84', 'no', '159', 'small', '234', 'story\n10', 'that', '85', 'most', '160', 'end', '235', 'saw\n11', 'he', '86', 'number', '161', 'put', '236', 'far\n12', 'was', '87', 'who', '162', 'home', '237', 'sea\n13', 'for', '88', 'over', '163', 'read', '238', 'draw\n14', 'on', '89', 'know', '164', 'hand', '239', 'left\n15', 'are', '90', 'water', '165', 'port', '240', 'late\n16', 'with', '91', 'than', '166', 'large', '241', 'run\n17', 'as', '92', 'call', '167', 'spell', '242', "don't\n18", 'I', '93', 'first', '168', 'add', '243', 'while\n19', 'his', '94', 'pe

In [31]:
# 단어 리스트에서 숫자를 모두 지운다

vocab_list = []
for item in data_into_list:
    newstring = ''.join([i for i in item if not i.isdigit()])
    newstring = newstring.strip('\n')
    if newstring:
        vocab_list.append(newstring)

In [ ]:
vocab_list

### 불러온 영어 단어 리스트에서 pos tagging을 통해 적절한 단어만 선택하기

In [33]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [34]:
pos_vocab = nltk.pos_tag(vocab_list)

In [ ]:
pos_vocab

In [36]:
filtered_vocab = []
for word, pos in pos_vocab:
    if pos in ['DT', 'IN', 'TO', 'MD', 'CC', 'PRP', 'PRP$']:  # 이 pos tag를 가진 단어는 제거
        pass
    else:
        filtered_vocab.append(word)
print(filtered_vocab)

['more', 'sentence', 'day', 'set', 'city', 'three', 'tree', 'go', 'want', 'cross', 'come', 'air', 'did', 'well', 'hard', 'is', 'also', 'start', 'sound', 'play', 'small', 'story', 'most', 'end', 'saw', 'number', 'put', 'far', 'was', 'who', 'home', 'sea', 'read', 'draw', 'know', 'hand', 'left', 'are', 'water', 'port', 'late', 'large', 'run', 'call', 'spell', "don't", 'first', 'add', 'people', 'even', 'land', 'close', 'be', 'down', 'here', 'night', 'side', 'real', 'one', 'been', 'big', 'life', 'have', 'now', 'high', 'few', 'find', 'such', 'stop', 'follow', 'open', 'new', 'act', 'seem', 'had', 'work', 'why', 'together', 'part', 'ask', 'next', 'hot', 'take', 'men', 'white', 'get', 'change', 'children', 'place', 'went', 'begin', 'what', 'made', 'light', 'got', 'there', 'live', 'kind', 'walk', 'where', 'off', 'example', 'need', 'ease', 'out', 'back', 'house', 'paper', 'other', 'little', 'picture', 'often', 'were', 'only', 'try', 'always', 'round', 'music', 'man', 'again', 'when', 'year', 'ani

In [37]:
en_vocabs = filtered_vocab

## template fine-class에 대한 단어 대입 함수 만들기

In [38]:
test_vocab = filtered_vocab[:5]

In [40]:
test_df = df.copy()
for i in range(30, 50):
    target = df['target'][i]
    if test_df['fine_class'][i]=='E2E':
        for j in range(len(test_vocab)):
            new_stc = df['text'][i].replace(target, test_vocab[j])
            print(new_stc)

How do you use more in a sentence?
How do you use sentence in a sentence?
How do you use day in a sentence?
How do you use set in a sentence?
How do you use city in a sentence?
How do you use the more in the sentence?
How do you use the sentence in the sentence?
How do you use the day in the sentence?
How do you use the set in the sentence?
How do you use the city in the sentence?
How do you use the more in the same sentence?
How do you use the sentence in the same sentence?
How do you use the day in the same sentence?
How do you use the set in the same sentence?
How do you use the city in the same sentence?
Can you use more in a sentence?
Can you use sentence in a sentence?
Can you use day in a sentence?
Can you use set in a sentence?
Can you use city in a sentence?
Can you use the more in the sentence?
Can you use the sentence in the sentence?
Can you use the day in the sentence?
Can you use the set in the sentence?
Can you use the city in the sentence?


### E2E, E2K 사용법

In [43]:
# E2E, E2K template 사용 방법
# E2E와 E2K는 조사가 없기 때문에 그냥 영어 단어를 대입하면 된다. 

def E2_insert(data, vocab_list):
    target, fine_class, text = data['target'], data['fine_class'], data['text']
    result = []
    for vocab in vocab_list:
        new_stc = text.replace(target, vocab)
        result.append((new_stc, vocab))
    return result

E2_insert(df.loc[30], test_vocab)              

[('What is more in Korean?', 'more'),
 ('What is sentence in Korean?', 'sentence'),
 ('What is day in Korean?', 'day'),
 ('What is set in Korean?', 'set'),
 ('What is city in Korean?', 'city')]

### 조사의 영향을 받는 영단어들을 처리하기 위해 영단어 리스트 분리

In [69]:
pip install hangulize

In [70]:
from hangulize import hangulize

In [71]:
hangulize('dog', 'cym')

'도그'

In [72]:
def split_en_vocabs(vocab_list):  # 영단어를 받침이 있는 단어들과 없는 단어들로 나누는 함수
    vocabs_neat, vocabs_dirty = [], []
    for vocab in vocab_list:
        if ends_with_jong(hangulize(vocab, 'cym')): # 영어 vocab도 받침이 있으면
            vocabs_dirty.append(vocab)
        else:
            vocabs_neat.append(vocab)
    return vocabs_neat, vocabs_dirty

In [73]:
len(en_vocabs)

232

In [74]:
en_vocabs_neat, en_vocabs_dirty = split_en_vocabs(en_vocabs)

In [75]:
print(len(en_vocabs_neat), len(en_vocabs_dirty))

177 55


### M2E_kor 사용법

In [ ]:
test_df = df.copy()
for i in range(16, 30):
    target = df['target'][i]
    if test_df['fine_class'][i]=='M2E':
        if isKorean(target):
            if is_postposition(df['text'][i], target)=='의':  # 템플릿에서 타겟 단어 뒤에 조사 '의'가 붙으면
                for j in range(len(test_vocab)):
                    new_stc = df['text'][i].replace(target, test_vocab[j])
                    print(new_stc)
                continue
            if ends_with_jong(target):  # 타겟 단어가 받침이 있으면
                for j in range(len(test_vocab)):
                    if ends_with_jong(hangulize(test_vocab[j], 'cym')): # 영어 vocab도 받침이 있으면
                        new_stc = df['text'][i].replace(target, test_vocab[j])
                        print(new_stc)
            else:  # 타겟 단어가 받침이 없으면
                for j in range(len(test_vocab)):
                    if not ends_with_jong(hangulize(test_vocab[j], 'cym')): # 영어 vocab도 받침이 없으면
                        new_stc = df['text'][i].replace(target, test_vocab[j])
                        print(new_stc)

In [ ]:
# M2E중 문장이 한글인 template 사용법
# 한글 조사에 따라 영단어를 구분해줘야 한다. 

def M2E_kor_insert(data, vocab_list, vocabs_neat, vocabs_dirty):
    target, fine_class, text = data['target'], data['fine_class'], data['text']
    result = []
    if is_postposition(text, target) == "의" or not is_postposition(text, target):
        # 조사가 "의"거나 없으면 어떤 영단어가 와도 무방
        for vocab in vocab_list:
            new_stc = text.replace(target, vocab)
            result.append((new_stc, vocab))
        return result

    if ends_with_jong(target):  
        for dirty_vocab in vocabs_dirty:  # 받침이 있는 영단어들의 리스트
            new_stc = text.replace(target, dirty_vocab)
            result.append((new_stc, dirty_vocab))
        return result

    else:
        for neat_vocab in vocabs_neat:
            new_stc = text.replace(target, neat_vocab)  # 받침이 없는 영단어들의 리스트
            result.append((new_stc, neat_vocab))
        return result

M2E_kor_insert(df.loc[26], en_vocabs, en_vocabs_neat, en_vocabs_dirty)

### 한글 단어 리스트 불러오기

In [54]:
kor_vocab_text = open("kor_vocab.txt", "r", encoding='cp949')  # https://goodmorningside.tistory.com/entry/초등학교-필수-영단어-800개-모음교육부자료
  
# reading the file
data = kor_vocab_text.read()
# replacing end splitting the text 
# when newline ('\n') is seen.
data_into_list = data.split(" \t")
print(data_into_list)
kor_vocab_text.close()

['1', 'ride', '타기\n2', 'sea', '바다\n3', 'side', '측면\n4', 'some', '약간\n5', 'steam', '증기\n6', 'surprise', '놀라다\n7', 'the', '그만큼\n8', 'tonight', '오늘 밤\n9', 'usual', '보통의\n10', 'welcome', '어서 오십시오\n11', 'write', '쓰다\n12', 'air', '공기\n13', 'back', '뒤\n14', 'beside', '빗나가서\n15', 'brown', '갈색\n16', 'care', '케어\n17', 'climb', '상승\n18', 'cream', '크림\n19', 'do', '하다\n20', 'empty', '빈\n21', 'father', '아버지\n22', 'fork', '포크\n23', 'grape', '포도\n24', 'heart', '심장\n25', 'hundred', '백\n26', 'kick', '발 차기\n27', 'lesson', '교훈\n28', 'mail', '우편\n29', 'month', '달\n30', 'night', '밤\n31', 'other', '다른\n32', 'pig', '돼지\n33', 'pull', '손잡이\n34', 'right', '권리\n35', 'season', '시즌\n36', 'sight', '시각\n37', 'son', '아들\n38', 'step', '단계\n39', 'sweater', '스웨터\n40', 'then', '그때\n41', 'too', '너무\n42', 'vacation', '휴가\n43', 'well', '잘\n44', 'wrong', '잘못된\n45', 'airplane', '비행기\n46', 'bad', '나쁜\n47', 'between', '중에서\n48', 'brush', '브러시\n49', 'carpenter', '목수\n50', 'clock', '시계\n51', 'cross', '가로 질러 가다\n52', 'doctor', '박사님

In [55]:
kor_vocabs = []
for item in data_into_list:
    newstring = ''.join([i for i in item if not i.isdigit()])
    newstring = newstring.strip('\n')
    if newstring and isKorean(newstring):
        kor_vocabs.append(newstring)

In [56]:
# 영단어와 개수를 맞춰주기 위해 랜덤하게 단어 200개 선택

import random
kor_vocabs = random.choices(kor_vocabs, k=200)
print(kor_vocabs)

['상자', '빨간', '댄스', '건너서', '승리', '설명하다', '음악', '저녁', '머리', '저녁 식사', '스위치', '팔다', '손', '마을', '주기', '또는', '운', '고치다', '봄', '그림', '클럽', '노래', '두번', '앞', '댄스', '달러', '타격', '분', '댄스', '오늘 밤', '상자', '마을', '미터', '스웨터', '백', '공항', '아버지', '함께', '보통의', '시즌', '상자', '병든', '접촉', '가르치다', '이해하다', '무거운', '오늘', '교회에', '이', '바쁜', '주사위', '전화', '단계', '운영', '표', '노랑', '교훈', '공기', '하느님', '가스', '신전', '종료', '선택', '멋있는', '바닷가', '하여야 한다', '시험', '테니스', '화려한', '비누', '베이스', '운', '그곳에', '네', '줄자', '음주', '사격', '...에서', '컴퓨터', '시', '대낮', '시각', '몸', '시', '떨어져', '우표', '샐러드', '가위', '요구', '장갑', '별', '보내다', '브러시', '요리', '저녁 식사', '울음 소리', '저녁 식사', '우유', '단단한', '질문', '알고있다', '드라이브', '슈퍼마켓', '죽은', '접촉', '타기', '놓다', '다만', '뜨거운', '빨간', '포인트', '표', '호텔', '그릇된', '호랑이', '크기', '...에', '안녕', '부드러운', '전화', '선장', '설탕', '팔다', '뭐', '일', '캠프', '가로 질러 가다', '광장', '말하다', '깨끗한', '마른', '기호', '공식 만찬', '상단', '중지', '얻다', '왕', '아니', '책', '필요한 것', '닫은', '장난', '후', '달', '서비스', '근처에', '생신', '낮은', '회전', '연기', '예', '부인', '앉다', '열다', '함께', '쿡', '슈퍼마켓',

### 조사 영향을 받는 단어 처리하기 위해 한글 단어 리스트 분리

In [57]:
def split_kor_vocab(vocab_list):
    vocabs_neat, vocabs_dirty = [], []
    for vocab in vocab_list:
        if ends_with_jong(vocab): # 한글 단어에 받침이 있으면
            vocabs_dirty.append(vocab)
        else:
            vocabs_neat.append(vocab)
    return vocabs_neat, vocabs_dirty

In [59]:
kor_vocabs_neat, kor_vocabs_dirty = split_kor_vocab(kor_vocabs)

In [60]:
print(len(kor_vocabs), len(kor_vocabs_neat), len(kor_vocabs_dirty))

200 106 94


### K2E 사용법

In [ ]:
# K2E template 사용 방법
# 마찬가지로 조사에 따라 대입하는 한글 단어 구분

def K2E_insert(data, vocab_list, vocabs_neat, vocabs_dirty):
    target, fine_class, text = data['target'], data['fine_class'], data['text']
    result = []
    if is_postposition(text, target) == "의" or not is_postposition(text, target):
        for vocab in vocab_list:
            new_stc = text.replace(target, vocab)
            result.append((new_stc, vocab))
        return result

    if ends_with_jong(target):
        for dirty_vocab in vocabs_dirty:  # 받침이 있는 영단어들의 리스트
            new_stc = text.replace(target, dirty_vocab)
            result.append((new_stc, dirty_vocab))
        return result

    else:
        for neat_vocab in vocabs_neat:
            new_stc = text.replace(target, neat_vocab)  # 받침이 없는 영단어들의 리스트
            result.append((new_stc, neat_vocab))
        return result

K2E_insert(df.loc[3], kor_vocabs, kor_vocabs_neat, kor_vocabs_dirty)

In [ ]:
# M2E_en template 사용법
# 조사가 없기 때문에 어떤 단어나 넣어도 무방

def M2E_en_insert(data, vocab_list):
    target, fine_class, text = data['target'], data['fine_class'], data['text']
    result = []
    for vocab in vocab_list:
        new_stc = text.replace(target, vocab)
        result.append((new_stc, vocab))
    return result

M2E_en_insert(df.loc[28], kor_vocabs) 

## 앞에서 만든 insert 함수들로 실제 dataframe을 변형한다

In [63]:
len(df)

127

In [64]:
df.loc[:]

,class,fine_class,text,target
0,TRS,K2E,영어로 고양이가 뭐에요?,고양이
1,TRS,K2E,영어로 고양이는 무엇인가?,고양이
2,TRS,K2E,영어로 고양이는 뭐야?,고양이
3,TRS,K2E,영어로 고양이가 뭐야?,고양이
4,TRS,K2E,영어로 기린이 뭐에요?,기린
...,...,...,...,...
122,RPT,K2E,잘 못 들었어요.,RPT
123,RPT,K2E,반복해 주실 수 있나요,RPT
124,RPT,K2E,반복해 주시겠어요?,RPT
125,RPT,K2E,반복해 주시겠습니까?,RPT


In [167]:
test_df = df.sample(n=20)

In [168]:
test_df.index=range(len(test_df))
test_df

,class,fine_class,text,target
0,RPT,K2E,다시 말씀해 주시겠어요?,RPT
1,STC,E2E,How do you use the cat in the same sentence?,cat
2,RPT,K2E,반복해 주시겠어요?,RPT
3,SPL,M2E,고양이의 철자는 무엇인가?,고양이
4,SPL,E2E,How do you spell the cat?,cat
5,TRS,K2E,기린은 영어로 뭐야?,기린
6,PRN,E2E,Can you pronounce cat?,cat
7,RPT,K2E,잘 못 들었어요.,RPT
8,RPT,E2E,Excuse me.,RPT
9,TRS,M2E,고양이의 의미가 뭐에요?,고양이


In [204]:
# M2E_en, M2E_kor, K2E, E2


new_test_df = pd.DataFrame()
for _, row in test_df.iterrows():
    cls, fine_class, text, target = row['class'], row['fine_class'], row['text'], row['target']

    if fine_class == 'K2E' and cls != 'RPT':
        new_stcs = K2E_insert(row, kor_vocabs, kor_vocabs_neat, kor_vocabs_dirty)
        for stc, new_word in new_stcs:
            copy = row.copy()
            copy['text'] = stc
            copy['target'] = new_word
            new_test_df = new_test_df.append(copy, ignore_index=True)
        continue

    if fine_class == 'E2E' or fine_class == 'E2K':
        new_stcs = E2_insert(row, en_vocabs)
        for stc, new_word in new_stcs:
            copy = row.copy()
            copy['text'] = stc
            copy['target'] = new_word
            new_test_df = new_test_df.append(copy, ignore_index=True)
        continue  

    if fine_class == 'M2E' and isKorean(target):
        new_stcs = M2E_kor_insert(row, en_vocabs, en_vocabs_neat, en_vocabs_dirty)
        for stc, new_word in new_stcs:
            copy = row.copy()
            copy['text'] = stc
            copy['target'] = new_word
            new_test_df = new_test_df.append(copy, ignore_index=True)
        continue  

    if fine_class == 'M2E' and not isKorean(target):
        new_stcs = M2E_en_insert(row, kor_vocabs)
        for stc, new_word in new_stcs:
            copy = row.copy()
            copy['text'] = stc
            copy['target'] = new_word
            new_test_df = new_test_df.append(copy, ignore_index=True)
        continue 


In [65]:
def append_series_to_dataframe(row, new_stcs, new_df): 
    # (generated sentence, new target word) 쌍을 입력받아 새로운 행 추가
    for stc, new_word in new_stcs:
        copy = row.copy()
        copy['text'] = stc
        copy['target'] = new_word
        new_df = new_df.append(copy, ignore_index=True)
    return new_df

In [66]:
def insert_words_to_template(template, en_vocabs, kor_vocabs):
    en_vocabs_neat, en_vocabs_dirty = split_en_vocabs(en_vocabs)
    kor_vocabs_neat, kor_vocabs_dirty = split_kor_vocab(kor_vocabs)
    new_df = pd.DataFrame()

    for _, row in tqdm(template.iterrows()):
        cls, fine_class, text, target = row['class'], row['fine_class'], row['text'], row['target']

        if fine_class == 'K2E' and cls != 'RPT':
            new_stcs = K2E_insert(row, kor_vocabs, kor_vocabs_neat, kor_vocabs_dirty)
            new_df = append_series_to_dataframe(row, new_stcs, new_df)
            continue

        if fine_class == 'E2E' or fine_class == 'E2K':
            if cls != 'RPT':
                new_stcs = E2_insert(row, en_vocabs)
                new_df = append_series_to_dataframe(row, new_stcs, new_df)
                continue  

        if fine_class == 'M2E' and isKorean(target):
            new_stcs = M2E_kor_insert(row, en_vocabs, en_vocabs_neat, en_vocabs_dirty)
            new_df = append_series_to_dataframe(row, new_stcs, new_df)
            continue  

        if fine_class == 'M2E' and not isKorean(target):
            new_stcs = M2E_en_insert(row, kor_vocabs)
            new_df = append_series_to_dataframe(row, new_stcs, new_df)
            continue 

        if cls == 'RPT':
            copy = row.copy()
            new_df = new_df.append(copy, ignore_index=True)
            continue

    return new_df

In [67]:
new_df = insert_words_to_template(df, en_vocabs, kor_vocabs)
new_df.to_csv("dataset.csv")

127it [00:47,  2.66it/s]


In [68]:
len(new_df)

16758

In [222]:
new_df.loc[16000:]

,class,fine_class,text,target
16000,SPL,M2E,head의 철자는 무엇인가.,head
16001,SPL,M2E,care의 철자는 무엇인가.,care
16002,SPL,M2E,do의 철자는 무엇인가.,do
16003,SPL,M2E,very의 철자는 무엇인가.,very
16004,SPL,M2E,stand의 철자는 무엇인가.,stand
...,...,...,...,...
16863,RPT,K2E,잘 못 들었어요.,RPT
16864,RPT,K2E,반복해 주실 수 있나요,RPT
16865,RPT,K2E,반복해 주시겠어요?,RPT
16866,RPT,K2E,반복해 주시겠습니까?,RPT
